### Importing Libaries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,roc_auc_score,roc_curve

### Import the data

In [3]:
df=pd.read_csv("C:/Users/Nantha/Desktop/Nantha Kumar/Capstone/train.csv")

In [4]:
def cns_desc(x):
    if x<300:
        return 0
    elif (x>=300) and (x<=350):
        return 1
    elif (x>350) and (x<=570):
        return 2
    elif (x>570) and (x<=630):
        return 3
    elif (x>630) and (x<=705):
        return 4
    else:
        return 5

In [5]:
df["Date.of.Birth"]=pd.to_datetime(df["Date.of.Birth"])

df["DisbursalDate"]=pd.to_datetime(df["DisbursalDate"])
df["DisbursalDate_Month"]=df["DisbursalDate"].dt.month

df["Age_at_disbursement"]=((df['DisbursalDate']-df["Date.of.Birth"])/365).apply(lambda x:x.days)

df["Employment.Type"]=df["Employment.Type"].map({'Salaried':0,"Self employed":1,np.nan:2})

df["PERFORM_CNS.SCORE.DESCRIPTION"]=df["PERFORM_CNS.SCORE"].apply(cns_desc)

df["AVERAGE.ACCT.AGE"]=df["AVERAGE.ACCT.AGE"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

df["CREDIT.HISTORY.LENGTH"]=df["CREDIT.HISTORY.LENGTH"].apply(lambda x:(int((x.split("y"))[0])*12)+(int((x.split())[1].split("m")[0])))

In [6]:
df['NO.OF.ACCTS']=df['PRI.NO.OF.ACCTS']+df['SEC.NO.OF.ACCTS']
df['ACTIVE.ACCTS']=df['PRI.ACTIVE.ACCTS']+df['SEC.ACTIVE.ACCTS']
df['OVERDUE.ACCTS']=df['PRI.OVERDUE.ACCTS']+df['SEC.OVERDUE.ACCTS']
df['CURRENT.BALANCE']=df['PRI.CURRENT.BALANCE']+df['SEC.CURRENT.BALANCE']
df['SANCTIONED.AMOUNT']=df['PRI.SANCTIONED.AMOUNT']+df['SEC.SANCTIONED.AMOUNT']
df['DISBURSED.AMOUNT']=df['PRI.DISBURSED.AMOUNT']+df['SEC.DISBURSED.AMOUNT']
df['INSTAL.AMT']=df['PRIMARY.INSTAL.AMT']+df['SEC.INSTAL.AMT']

df=df.drop(['PRI.NO.OF.ACCTS','SEC.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','SEC.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','SEC.OVERDUE.ACCTS','PRI.CURRENT.BALANCE','SEC.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','SEC.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT'],axis=1)

In [7]:
df.nunique()

UniqueID                               233154
disbursed_amount                        24565
asset_cost                              46252
ltv                                      6579
branch_id                                  82
supplier_id                              2953
manufacturer_id                            11
Current_pincode_ID                       6698
Date.of.Birth                           15433
Employment.Type                             3
DisbursalDate                              84
State_ID                                   22
Employee_code_ID                         3270
MobileNo_Avl_Flag                           1
Aadhar_flag                                 2
PAN_flag                                    2
VoterID_flag                                2
Driving_flag                                2
Passport_flag                               2
PERFORM_CNS.SCORE                         573
PERFORM_CNS.SCORE.DESCRIPTION               6
NEW.ACCTS.IN.LAST.SIX.MONTHS      

In [8]:
df.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AVERAGE.ACCT.AGE',
       'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES', 'loan_default',
       'DisbursalDate_Month', 'Age_at_disbursement', 'NO.OF.ACCTS',
       'ACTIVE.ACCTS', 'OVERDUE.ACCTS', 'CURRENT.BALANCE', 'SANCTIONED.AMOUNT',
       'DISBURSED.AMOUNT', 'INSTAL.AMT'],
      dtype='object')

In [9]:
df=df.drop(columns=['Date.of.Birth',"DisbursalDate"])

In [10]:
cols=df.columns.to_list()

In [11]:
cat_cols=['UniqueID','branch_id','supplier_id', 'manufacturer_id', 'Current_pincode_ID','Employment.Type', 'State_ID',
          'Employee_code_ID','MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag','Driving_flag', 'Passport_flag',
       'PERFORM_CNS.SCORE.DESCRIPTION','DisbursalDate_Month']

In [12]:
target_col="loan_default"

In [13]:
num_cols=['disbursed_amount','asset_cost','ltv','PERFORM_CNS.SCORE','NEW.ACCTS.IN.LAST.SIX.MONTHS',
          'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH','NO.OF_INQUIRIES',
          'Age_at_disbursement','NO.OF.ACCTS','ACTIVE.ACCTS','OVERDUE.ACCTS','CURRENT.BALANCE','SANCTIONED.AMOUNT',
             'DISBURSED.AMOUNT','INSTAL.AMT']

In [14]:
df[cat_cols].nunique()

UniqueID                         233154
branch_id                            82
supplier_id                        2953
manufacturer_id                      11
Current_pincode_ID                 6698
Employment.Type                       3
State_ID                             22
Employee_code_ID                   3270
MobileNo_Avl_Flag                     1
Aadhar_flag                           2
PAN_flag                              2
VoterID_flag                          2
Driving_flag                          2
Passport_flag                         2
PERFORM_CNS.SCORE.DESCRIPTION         6
DisbursalDate_Month                  12
dtype: int64

As we see no of unique elements in some categorical columns is higher,we couldn't take dummies as it will cause curse of 
dimensionality.Let's drop them.

In [15]:
linear_models_df=df.copy()

In [16]:
linear_models_df=linear_models_df.drop(columns=['UniqueID','supplier_id','Current_pincode_ID','Employee_code_ID',"MobileNo_Avl_Flag"])

In [17]:
cols_to_be_dummied=["branch_id","State_ID","manufacturer_id",'Employment.Type','DisbursalDate_Month','PERFORM_CNS.SCORE.DESCRIPTION']

In [18]:
linear_models_final_df=pd.get_dummies(data=linear_models_df,columns=cols_to_be_dummied,drop_first=True)

In [20]:
linear_models_df.corr()['loan_default']

disbursed_amount                       0.077675
asset_cost                             0.014261
ltv                                    0.098208
branch_id                              0.030193
manufacturer_id                       -0.025039
Employment.Type                        0.024823
State_ID                               0.048075
Aadhar_flag                           -0.041593
PAN_flag                               0.002046
VoterID_flag                           0.043747
Driving_flag                          -0.005821
Passport_flag                         -0.007602
PERFORM_CNS.SCORE                     -0.057929
PERFORM_CNS.SCORE.DESCRIPTION         -0.067798
NEW.ACCTS.IN.LAST.SIX.MONTHS          -0.029400
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS    0.034462
AVERAGE.ACCT.AGE                      -0.024781
CREDIT.HISTORY.LENGTH                 -0.042126
NO.OF_INQUIRIES                        0.043678
loan_default                           1.000000
DisbursalDate_Month                    0

**Since Current balance has negative values we will take scaled between 0 and 1**

In [21]:
scalar=MinMaxScaler()
linear_models_final_df["CURRENT.BALANCE"]=scalar.fit_transform(linear_models_final_df["CURRENT.BALANCE"].values.reshape(-1,1))

In [22]:
val=[]
for i in num_cols:
    val.append(linear_models_df[i].skew())
skew_df=pd.DataFrame(index=num_cols,data=val,columns=["Scores"])
skew_df

,Scores
disbursed_amount,4.492240
asset_cost,6.133485
ltv,-1.075766
PERFORM_CNS.SCORE,0.445150
NEW.ACCTS.IN.LAST.SIX.MONTHS,4.839326
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,6.641996
AVERAGE.ACCT.AGE,3.285142
CREDIT.HISTORY.LENGTH,2.969155
NO.OF_INQUIRIES,7.870683
Age_at_disbursement,-2.250138


In [23]:
cols_to_taken_log=skew_df[skew_df["Scores"]>2].index

In [24]:
for i in cols_to_taken_log:
    linear_models_final_df[i]=np.log(linear_models_final_df[i]+1)

In [25]:
X=linear_models_final_df.drop('loan_default',axis=1)
y=linear_models_final_df['loan_default']
model_scores={}

In [26]:
linear_models_final_df.head()

,disbursed_amount,asset_cost,ltv,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,NEW.ACCTS.IN.LAST.SIX.MONTHS,...,DisbursalDate_Month_8,DisbursalDate_Month_9,DisbursalDate_Month_10,DisbursalDate_Month_11,DisbursalDate_Month_12,PERFORM_CNS.SCORE.DESCRIPTION_1,PERFORM_CNS.SCORE.DESCRIPTION_2,PERFORM_CNS.SCORE.DESCRIPTION_3,PERFORM_CNS.SCORE.DESCRIPTION_4,PERFORM_CNS.SCORE.DESCRIPTION_5
0,10.831292,10.975088,89.55,1,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10.761004,11.090584,73.23,1,0,0,0,0,598,0.0,...,0,1,0,0,0,0,0,1,0,0
2,10.883298,11.024530,89.63,1,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10.959784,11.099136,88.48,1,0,0,0,0,305,0.0,...,0,0,1,0,0,1,0,0,0,0
4,10.866261,11.007104,88.39,1,0,0,0,0,0,0.0,...,0,1,0,0,0,0,0,0,0,0


# Model Building

In [27]:
#model selection from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#ensemble
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
#metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [31]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [37]:
y_train.shape

(163207,)

In [39]:
X_test.shape,y_test.shape

((69947, 152), (69947,))

In [53]:
def modelresult(i):
    print(i,"\n")
    classifier = i
    classifier.fit(X_train, y_train)
    y_predt=classifier.predict(X_train)
    y_pred=classifier.predict(X_test)
    print("\n Accuracy for the Train:",accuracy_score(y_train,y_predt))
    print("\n Accuracy for the Test",accuracy_score(y_test,y_pred),"\n")
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_pred))

## Logistic Regression

In [54]:
lr = LogisticRegression(random_state=1)
modelresult(lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False) 


 Accuracy for the Train: 0.7828218152407679

 Accuracy for the Test 0.7829928374340572 

              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54776
           1       0.48      0.01      0.01     15171

    accuracy                           0.78     69947
   macro avg       0.63      0.50      0.44     69947
weighted avg       0.72      0.78      0.69     69947



## Decision Tree classifier

In [59]:
dcg = DecisionTreeClassifier(criterion='entropy',random_state=1)
modelresult(dcg)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best') 


 Accuracy for the Train: 0.9997365309086007

 Accuracy for the Test 0.6760976167669808 

              precision    recall  f1-score   support

           0       0.80      0.79      0.79     54776
           1       0.26      0.27      0.27     15171

    accuracy                           0.68     69947
   macro avg       0.53      0.53      0.53     69947
weighted avg       0.68      0.68      0.68     69947



In [60]:
dcg = DecisionTreeClassifier(criterion='gini',random_state=1)
modelresult(dcg)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best') 


 Accuracy for the Train: 0.9997365309086007

 Accuracy for the Test 0.6747823351966489 

              precision    recall  f1-score   support

           0       0.80      0.78      0.79     54776
           1       0.26      0.28      0.27     15171

    accuracy                           0.67     69947
   macro avg       0.53      0.53      0.53     69947
weighted avg       0.68      0.67      0.68     69947



## RandomForest Classifier

In [62]:
rfc = RandomForestClassifier(n_estimators=100,random_state=1)
modelresult(rfc)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False) 


 Accuracy for the Train: 0.9997120221559125

 Accuracy for the Test 0.7771598496004117 

              precision    recall  f1-score   support

           0       0.79      0.98      0.87     54776
           1       0.39      0.05      0.08     15171

    accuracy                           0.78     69947
   macro avg       0.59      0.51      0.48     69947
weighted avg       0.70      0.78      0.70     69947



## Bagging Classifier

In [63]:
bgc = BaggingClassifier(random_state=1)
modelresult(bgc)  

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=1, verbose=0,
                  warm_start=False) 


 Accuracy for the Train: 0.9750868528923392

 Accuracy for the Test 0.7650506812300742 

              precision    recall  f1-score   support

           0       0.79      0.95      0.86     54776
           1       0.34      0.09      0.14     15171

    accuracy                           0.77     69947
   macro avg       0.56      0.52      0.50     69947
weighted avg       0.69      0.77      0.71     69947



## AdaBoost Classifier

In [64]:
ada = AdaBoostClassifier(random_state=1)
modelresult(ada)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=1) 


 Accuracy for the Train: 0.7827115258536705

 Accuracy for the Test 0.7826783135802822 

              precision    recall  f1-score   support

           0       0.79      0.99      0.88     54776
           1       0.47      0.02      0.03     15171

    accuracy                           0.78     69947
   macro avg       0.63      0.51      0.45     69947
weighted avg       0.72      0.78      0.69     69947



## XGBoost Classifier

In [65]:
xgb =  XGBClassifier(random_state=1, n_jobs=-1, learning_rate=0.2,n_estimators=100, max_depth=3)
modelresult(xgb)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1) 


 Accuracy for the Train: 0.7838328012891604

 Accuracy for the Test 0.783693367835647 

              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54776
           1       0.57      0.01      0.02     15171

    accuracy                           0.78     69947
   macro avg       0.68      0.50      0.45     69947
weighted avg       0.74      0.78      0.69     69947



We can observe that Xgboost classifier has highest accuracy score of 78.3% out of all models built.

In [42]:
# def validation(X,y,model,cv=230,scoring="roc_auc"):
#     results=cross_val_score(model,X=X,y=y,cv=cv,scoring=scoring,n_jobs=-1)
#     print("mean_auc-",results.mean())
#     print("std_auc-",results.std())
#     return results

In [41]:
# df.shape

(233154, 34)

### LogisticRegression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# model=LogisticRegression()
# results=validation(X,y,model)
# model_scores.update({"Logistic_Regression":results})

In [29]:
# model_scores

{'Logistic_Regression': array([0.64332927, 0.63215232, 0.64009764, 0.6190023 , 0.63048149,
        0.63910918, 0.63761993])}

In [30]:
# results=validation(X,y,model,cv=10)

mean_auc- 0.6309277182740594
std_auc- 0.014406586730994566


In [31]:
# model_scores.update({"Logistic_Regression_cv10":results})

In [32]:
# model_scores

{'Logistic_Regression': array([0.64332927, 0.63215232, 0.64009764, 0.6190023 , 0.63048149,
        0.63910918, 0.63761993]),
 'Logistic_Regression_cv10': array([0.6407164 , 0.64764169, 0.62044767, 0.64307746, 0.62032089,
        0.61465774, 0.60614936, 0.64954661, 0.6265333 , 0.64018604])}

### LinearDiscriminantAnalysis

In [33]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# model=LinearDiscriminantAnalysis()

In [27]:
# results=validation(X,y,model)
# model_scores.update({"LinearDiscriminantAnalysis":results})

mean_auc- 0.639158972950011
std_auc- 0.010641886010019763


## Tree Based Models:

In [45]:
# df1=df.copy()
# # df1=df1.drop(columns=["UniqueID"])

In [35]:

# df1[['supplier_id','Current_pincode_ID','Employee_code_ID']].nunique()

supplier_id           2953
Current_pincode_ID    6698
Employee_code_ID      3270
dtype: int64

In [46]:
# from sklearn.preprocessing import OneHotEncoder
# for i in ['supplier_id','Current_pincode_ID','Employee_code_ID']:
#     one_hot=OneHotEncoder(handle_unknown="ignore",sparse=False)
#     dummy_index=df1[i].value_counts()[:100].index.to_list()
#     one_hot.fit(np.array(dummy_index).reshape(-1, 1))
#     data=one_hot.transform(df1[i].values.reshape(-1, 1))
#     dummy=pd.DataFrame(data=data,columns=[str(j)+i for j in dummy_index])
#     df1=pd.concat([df1,dummy],axis=1)

# df1=pd.get_dummies(data=df1,columns=cols_to_be_dummied,drop_first=True)

In [57]:
df1.shape

(233154, 457)

### Decision Tree

In [66]:
# Xd=df1.drop('loan_default',axis=1)
# yd=df1['loan_default']
# from sklearn.tree import DecisionTreeClassifier
# dtc_model=DecisionTreeClassifier()
# # results=validation(X,y,model,cv=10)
# # model_scores.update({"Decision_Tree":results})

In [67]:
# from sklearn.model_selection import train_test_split,KFold,cross_val_score
# X_traind, X_testd, y_traind, y_testd = train_test_split(Xd, yd, test_size=0.3, random_state=2)

In [68]:
# modelresult(dtc_model)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best') 


 Accuracy for the Train: 0.9997365309086007

 Accuracy for the Test 0.6738673567129398 

              precision    recall  f1-score   support

           0       0.80      0.78      0.79     54776
           1       0.26      0.28      0.27     15171

    accuracy                           0.67     69947
   macro avg       0.53      0.53      0.53     69947
weighted avg       0.68      0.67      0.68     69947

